<a href="https://colab.research.google.com/github/priyansh4320/tensorflow-practice/blob/main/SNS_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Similarity Classification Model

In [64]:
#installing necessary lib
!pip install transformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a467e8feb8eb375e4505486b382fe6cb1973f95ea4a22bb46299196b986b1a3a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [6]:
!pip install torch torchvision -U

In [62]:
#checking if torch is present
import torch

print(torch.__version__)

2.0.1+cu118


In [65]:
#import necessary lib
from sentence_transformers import SentenceTransformer, util
import numpy as np
from functools import cache

In [66]:
# load pretrained model stsb-roberta-large
# will be used to extract embeddings and Similarity score from text
model = SentenceTransformer('stsb-roberta-large')

In [67]:
#testing the pretrained model
sentence1 = "I like Python because I can build AI applications"
sentence2 = "I like Python because I can do data analytics"
# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

Sentence 1: I like Python because I can build AI applications
Sentence 2: I like Python because I can do data analytics
Similarity score: 0.8015277981758118


In [96]:
e = model.encode(df['text1'][0:5],convert_to_tensor=True)

In [13]:
#import dataset
import pandas as pd

In [14]:
df = pd.read_csv("/content/drive/MyDrive/Precily AI/Precily_Text_Similarity.csv")
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [20]:
#function for computing similarity score for each pair of text
@cache
def compute_similarity_score(dataframe , len_of_dataframe):
  # encode sentences to get their embeddings
  embedding1 = model.encode(sentence1, convert_to_tensor=True)
  embedding2 = model.encode(sentence2, convert_to_tensor=True)
# compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  return cosine_scores.item()

score = compute_similarity_score(sentence1,sentence2)
score

[0.25230738520622253,
 0.2531975209712982,
 0.1261926293373108,
 0.34469395875930786,
 0.38908863067626953,
 0.1271783858537674,
 0.11437475681304932,
 0.2231573760509491,
 0.1401197612285614,
 0.2657514214515686]

In [34]:
#compute similarity score for every pair
similarity_scores = []
for i in range(len(df)):
  similarity_scores.append(compute_similarity_score(df['text1'][i],df['text2'][i]))

similarity_score[:10]

[0.25230738520622253,
 0.2531975209712982,
 0.1261926293373108,
 0.34469395875930786,
 0.38908863067626953,
 0.1271783858537674,
 0.11437475681304932,
 0.2231573760509491,
 0.1401197612285614,
 0.2657514214515686]

In [40]:
len(similarity_scores)

3000

In [42]:
#Generating labels for training 1: similar , 0: not similar
result = []
for i in similarity_scores:
  if i>0.5:
    result.append(1)
  else:
    result.append(0)

In [45]:
len(result)

list

In [46]:
#append result and similarity to dataset
df['similarity'] = similarity_scores
df['result']     = result
df.head()

,text1,text2,similarity,result
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.252307,0
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.253198,0
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.126193,0
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.344694,0
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.389089,0


In [47]:
#save progress
df.to_csv("text_similarity2.csv")

In [81]:
#load progess
df = pd.read_csv("/content/drive/MyDrive/Precily AI/text_similarity2.csv")
df= df[['text1','text2','similarity','result']]
df.head()

,text1,text2,similarity,result
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.252307,0
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.253198,0
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.126193,0
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.344694,0
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.389089,0


In [87]:
#now extract embeddings
text1_embedding, text2_embedding = [],[]
df_len = len(df)

In [97]:

embedding1 = model.encode(df['text1'], convert_to_tensor=True)
embedding2 = model.encode(df['text2'], convert_to_tensor=True)


In [99]:
embedding1[:5]

tensor([[ 0.5764,  0.6705,  0.7088,  ...,  0.8344, -0.5563, -0.2122],
        [-0.6163,  0.8100,  0.1974,  ...,  0.3304, -0.8882, -0.1140],
        [ 0.4100,  1.0177,  0.4061,  ...,  0.3243, -0.4040, -0.3931],
        [-0.6264,  0.1350, -0.1653,  ...,  0.1603, -1.2940,  0.0064],
        [ 0.4911, -0.1716,  0.9481,  ...,  0.1972, -0.5469,  0.5312]])

In [100]:
#now concatinate embedding1 and embedding2 for training as X numpy array
X = np.concatenate([embedding1,embedding2],axis=1)
X[:5]

array([[ 0.57639503,  0.6705359 ,  0.7088014 , ...,  1.1659853 ,
        -0.54011905, -0.06296508],
       [-0.61634904,  0.81004906,  0.19742122, ..., -0.45929897,
        -1.0696697 ,  0.01065251],
       [ 0.4100224 ,  1.0177348 ,  0.40611142, ..., -0.17751044,
        -2.8012593 ,  0.8040325 ],
       [-0.6263867 ,  0.13501585, -0.16530204, ...,  0.66842824,
        -1.9837095 ,  0.3454319 ],
       [ 0.49110913, -0.17162094,  0.9481421 , ...,  0.35773054,
        -0.5529529 ,  0.77124524]], dtype=float32)

In [132]:
#generate probabilities for training data
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X = scaler.fit_transform(X)
X[:5]

array([[ 0.5971612 , -0.12236679,  1.3222721 , ...,  1.4483426 ,
         0.58429754, -0.33636683],
       [-1.207552  ,  0.09906124,  0.39740852, ..., -1.0287755 ,
        -0.10338709, -0.23091076],
       [ 0.34542662,  0.42868924,  0.77483803, ..., -0.5992977 ,
        -2.3520622 ,  0.90559393],
       [-1.2227397 , -0.97231656, -0.2585996 , ...,  0.690009  ,
        -1.2903764 ,  0.24865559],
       [ 0.46811706, -1.4589945 ,  1.755135  , ...,  0.21647039,
         0.56763124,  0.85862666]], dtype=float32)

In [134]:
# target variable
Y = np.asarray(df['result'])
y[:5]

array([0.25230739, 0.25319752, 0.12619263, 0.34469396, 0.38908863])

In [102]:
#import tensorflow lib for modelling
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split


In [135]:
#splitting the dataset
trainx , testx, trainy, testy = train_test_split(X,Y,test_size=0.2);

In [136]:
#calculating max_length of sequences in to pass it as parameter in Dense layer: input_dim
seq_len = [len(i) for i in X ]
max_sequence_length = max(seq_len)

In [137]:

#classification model logistic regression
mod = Sequential()
mod.add(Dense(1, input_dim= max_sequence_length, activation = 'sigmoid' ))
mod.compile(loss='binary_crossentropy',metrics = ["accuracy"], optimizer='rmsprop')
mod.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 1)                 2049      
                                                                 
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________


In [138]:
mod.fit(trainx,trainy, epochs = 100)

Epoch 1/100
75/75 [==============================] - 0s 1ms/step - loss: 0.8247 - accuracy: 0.4683
Epoch 2/100
75/75 [==============================] - 0s 950us/step - loss: 0.7211 - accuracy: 0.5425
Epoch 3/100
75/75 [==============================] - 0s 1ms/step - loss: 0.6711 - accuracy: 0.5925
Epoch 4/100
75/75 [==============================] - 0s 1ms/step - loss: 0.6269 - accuracy: 0.6521
Epoch 5/100
75/75 [==============================] - 0s 1ms/step - loss: 0.5906 - accuracy: 0.6933
Epoch 6/100
75/75 [==============================] - 0s 1ms/step - loss: 0.5524 - accuracy: 0.7342
Epoch 7/100
75/75 [==============================] - 0s 1ms/step - loss: 0.5231 - accuracy: 0.7817
Epoch 8/100
75/75 [==============================] - 0s 1ms/step - loss: 0.4919 - accuracy: 0.8229
Epoch 9/100
75/75 [==============================] - 0s 1ms/step - loss: 0.4669 - accuracy: 0.8529
Epoch 10/100
75/75 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.8750
Epoch 1

In [139]:
#predict test set
pred2 = mod.predict(testx)
pred2[:5]

19/19 [==============================] - 0s 888us/step


array([[0.00207215],
       [0.00024324],
       [0.00269941],
       [0.01192296],
       [0.00226603]], dtype=float32)

In [140]:
testy[:5]

array([0, 0, 0, 0, 0])

In [143]:
#evaluate model
mod.evaluate(testx,testy)

19/19 [==============================] - 0s 1ms/step - loss: 0.1972 - accuracy: 0.9517


[0.19722384214401245, 0.9516666531562805]

In [148]:
#classification model logistic regression all data
model = Sequential()
model.add(Dense(1, input_dim= max_sequence_length, activation = 'sigmoid' ))
model.compile(loss='binary_crossentropy',metrics = ["accuracy"], optimizer='rmsprop')
model.summary()
model.fit(X,Y, epochs=100)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 1)                 2049      
                                                                 
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
94/94 [==============================] - 0s 1ms/step - loss: 0.8117 - accuracy: 0.4810
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 0.7161 - accuracy: 0.5390
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.6588 - accuracy: 0.6097
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.6119 - accuracy: 0.6817
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5696 - accuracy: 0.7323
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5321 - ac

In [149]:
mod.save("precily_sns_mod_complete")

In [150]:
#saving the model
import shutil
from google.colab import files


folder_name = 'precily_sns_mod_complete'


shutil.make_archive(folder_name, 'zip', folder_name)


files.download(f'{folder_name}.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>